In [1]:
%matplotlib inline

# adding project dirs to path so code may be referenced from the notebook
import sys
sys.path.insert(0, '..')

In [2]:
import os
import config
# import const
import utils
# from hydrotools.nwm_client import gcp as nwm
from datetime import datetime, timedelta
# import pandas as pd
from usgs_to_parquet import fetch_usgs

In [4]:
start = datetime(2023, 1, 2)
download_period = timedelta(days=1)
number_of_periods = 11

sites = utils.get_usgs_gages()

# Fetch USGS gage data in daily batches
for p in range(number_of_periods):

    # Setup start and end date for fetch
    start_dt = (start + download_period * p)
    end_dt = (start + download_period * (p + 1))
    start_dt_str = start_dt.strftime("%Y-%m-%d")
    end_dt_str = end_dt.strftime("%Y-%m-%d")

    observations_data = fetch_usgs(
        sites=sites["gage_id"],
        start_dt=start_dt_str,
        end_dt=end_dt_str
    )

    # Filter out data not on the hour
    observations_data.set_index("value_time", inplace=True)
    obs = observations_data[
        observations_data.index.hour.isin(range(0, 23)) 
        & (observations_data.index.minute == 0) 
        & (observations_data.index.second == 0)
    ]
    obs.reset_index(level=0, allow_duplicates=True, inplace=True)

    # Save as parquet file
    parquet_filepath = os.path.join(config.USGS_PARQUET, f"{start_dt.strftime('%Y%m%d')}.parquet")
    utils.make_parent_dir(parquet_filepath)
    obs.to_parquet(parquet_filepath)

/srv/conda/envs/notebook/lib/python3.10/site-packages/hydrotools/nwis_client/iv.py:283: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfs.loc[:, "value"] = pd.to_numeric(dfs["value"], downcast="float")
/srv/conda/envs/notebook/lib/python3.10/site-packages/hydrotools/nwis_client/iv.py:283: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfs.loc[:, "value"] = pd.to_numeric(dfs["value"], downcast="float")
/srv/conda/envs/notebook/lib/python3.10/site-packages/hydrotools/nwis_client/iv.py:283: FutureWarning: In a future version, `df.iloc[:, i] = newvals` wi